In [172]:
import os
import sys

In [173]:
sys.path.append("C:\工具包")

In [174]:
import cvxopt
import toad
import sklearn2pmml

In [175]:
from hundredim.wrapper.experiment import score_card_experiment
from hundredim.detection import detection
from hundredim.util import analysis_tool
from hundredim.util import tools
import importlib
from hundredim.transform.binning import autobinning
# importlib.reload(autobinning)
from hundredim.wrapper import combiner
from hundredim.util import combine_scores
from hundredim.util.analysis_tool import cal_psi
from hundredim.report_generate import score_card_report_render
from hundredim.wrapper import score_card_report
import hundredim.util.tools
import pandas as pd
import numpy as np
import os
import sys
import gc
import re
import tqdm
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

In [176]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 10000)

In [177]:
target_all=pd.read_excel(r"D:\work\测评\飞鸟\测试数据_target_20250709_49W.xlsx")

In [178]:
cust_map = {'滴滴金融':'DD','NB':'NB','海尔':'HR','YG':'YG'}
target_all['customer'] = target_all['customer'].apply(lambda x: cust_map[x] if x in cust_map else x)
target_all['date'] = target_all['date'].apply(lambda x: str(x)[:10])
#target_all = target_all[target_all['zy_mth'].fillna(0).eq(0)]
target_all['irr_type'] = target_all['customer'].apply(lambda x: 'irr14' if x in ['BZ', 'EZ'] else 'irr24' if x in ['DR',] else 'irr36')

target_all.groupby(['customer', 'channel'])['target'].agg(['count','sum','mean'])

count   sum      mean
customer channel                        
DD       新客       175000  9024  0.051566
HR       Hirr     100000  5719  0.057190
         Lirr     134000  6492  0.048448
NB       字节        39705  1924  0.048457
YG       yqg       39685  4878  0.122918

In [179]:
target_all.groupby('customer')['target'].agg(['count','sum','mean'])

,count,sum,mean
customer,,,
DD,175000,9024,0.051566
HR,234000,12211,0.052184
NB,39705,1924,0.048457
YG,39685,4878,0.122918


In [180]:
target_all.head()

,dataset,id_no,phone,date,target,customer,channel,encrypt,irr_type
0,B,b58829676fbea04eaeeae56c454674ce39ab20406a7380...,b0a19689062d9f8d0293de9bc180baa8a58ae800f57189...,2024-03-25,0,DD,新客,SHA256,irr36
1,B,132921b6bde29486fafe3a83b4d19d5751fa0e1878bd48...,d282221b2909da4288ae2b7c4743fb7536b66bcd0a00ec...,2023-12-21,0,DD,新客,SHA256,irr36
2,B,18011c71b1001a2854923d7d0ceaeb02303451b7fdc8fe...,f991f0b30814e1abd0294b7745c3a180bb188198188018...,2023-08-16,0,DD,新客,SHA256,irr36
3,B,07fe4f828c5cf8fc7e44b175d3abf9367bed5620467bd0...,160978781420107f43f7e6ba57d9241d6e1d2447eeb303...,2023-12-04,0,DD,新客,SHA256,irr36
4,B,3c074449dc6568791e24267feeca479905fd1b1e214134...,7a5445ffb3f28f2b210f35f3c17ea7b444142aa6183b3f...,2024-03-03,1,DD,新客,SHA256,irr36


In [181]:
target_all['date'] = target_all['date'].apply(lambda x: str(x)[:10])
#target_all = target_all[target_all['zy_mth'].fillna(0).eq(0)]
#target_all['irr_type'] = target_all['customer'].apply(lambda x: 'irr14' if x in ['JX', 'NB'] else 'irr24' if x in ['DD','HR'] else 'irr36' if x in ['HC','LX','YG','YQG'] else None)

target_all.groupby(['customer', 'channel'])['target'].agg(['count','sum','mean'])

count   sum      mean
customer channel                        
DD       新客       175000  9024  0.051566
HR       Hirr     100000  5719  0.057190
         Lirr     134000  6492  0.048448
NB       字节        39705  1924  0.048457
YG       yqg       39685  4878  0.122918

In [182]:
target_all['date'] = (
    pd.to_datetime(target_all['date'], errors='coerce')
)
target_all['date'].head()

0   2024-03-25
1   2023-12-21
2   2023-08-16
3   2023-12-04
4   2024-03-03
Name: date, dtype: datetime64[ns]

In [183]:
target_all['phone'].duplicated().sum()

np.int64(0)

In [202]:
## 读取返回数据
raw_result=pd.read_csv(r"D:\work\测评\飞鸟\上海路诚测试样本_jktrack结果返回_20250725\上海路诚测试样本_jktrack结果返回_20250725\上海路诚样本_jktrack结果返回_20250725.csv")

In [203]:
raw_result['date'] = (
    pd.to_datetime(raw_result['date'], errors='coerce')
    .dt.strftime('%Y%m%d')
)
raw_result['date'].head()

0    20231128
1    20231209
2    20240209
3    20231102
4    20231125
Name: date, dtype: object

In [204]:
raw_result['date'] = raw_result['date'].apply(lambda x : str(x).replace('-',''))

In [205]:
raw_result['date'].head(2)

0    20231128
1    20231209
Name: date, dtype: object

In [206]:
target_all['phone'] = target_all['phone'].astype(str).str.strip()
raw_result['phone'] = raw_result['phone'].astype(str).str.strip()
target_all['phone'] = target_all['phone'].str.lower()
raw_result['phone'] = raw_result['phone'].str.lower()

In [187]:
mapping_phone=pd.read_csv(r"D:\work\测评\飞鸟\样本ID映射关系更新.csv")

In [264]:
print(target_all.shape)

(488390, 9)


In [207]:
mapping_phone['phone'] = mapping_phone['phone'].astype(str).str.strip()
mapping_phone['date'] = (
    pd.to_datetime(mapping_phone['date'], errors='coerce')
    .dt.strftime('%Y%m%d')
)
mapping_phone['date'].head()

0    20231216
1    20231216
2    20240209
3    20231124
4    20240110
Name: date, dtype: object

In [208]:
mapping_phone.drop_duplicates(subset=['uuid','date'],inplace=True)
raw_result.drop_duplicates(subset=['uuid','date'],inplace=True)

In [209]:
score_fn=mapping_phone.merge(raw_result,on=['uuid','date'],how='left')

In [210]:
score_fn.drop(columns=['phone_y','encrypt_y','uuid'],inplace=True)

In [211]:
score_fn.rename(columns={'phone_x':'phone','encrypt_x':'encrypt'},inplace=True)

In [212]:
score_fn.head(2)

,phone,date,encrypt,residence_city,level_code,level_name,poi_1466_visitDays_cnt_30d,poi_1466_visitDays_cnt_90d,poi_1466_visitDays_cnt_180d,poi_1466_visitPoi_cnt_30d,poi_1466_visitPoi_cnt_90d,poi_1466_visitPoi_cnt_180d,poi_1466_visitTime_dayavg_30d,poi_1466_visitTime_dayavg_90d,poi_1466_visitTime_dayavg_180d,poi_698_visitDays_cnt_30d,poi_698_visitDays_cnt_90d,poi_698_visitDays_cnt_180d,poi_698_visitPoi_cnt_30d,poi_698_visitPoi_cnt_90d,poi_698_visitPoi_cnt_180d,poi_698_visitTime_dayavg_30d,poi_698_visitTime_dayavg_90d,poi_698_visitTime_dayavg_180d,poi_1463_visitDays_cnt_30d,poi_1463_visitDays_cnt_90d,poi_1463_visitDays_cnt_180d,poi_1463_visitPoi_cnt_30d,poi_1463_visitPoi_cnt_90d,poi_1463_visitPoi_cnt_180d,poi_1463_visitTime_dayavg_30d,poi_1463_visitTime_dayavg_90d,poi_1463_visitTime_dayavg_180d,poi_191_visitDays_cnt_30d,poi_191_visitDays_cnt_90d,poi_191_visitDays_cnt_180d,poi_191_visitPoi_cnt_30d,poi_191_visitPoi_cnt_90d,poi_191_visitPoi_cnt_180d,poi_191_visitTime_dayavg_30d,poi_191_visitTime_dayavg_90d,poi_191_visitTime_dayavg_180d,poi_113_visitDays_cnt_30d,poi_113_visitDays_cnt_90d,poi_113_visitDays_cnt_180d,poi_113_visitPoi_cnt_30d,poi_113_visitPoi_cnt_90d,poi_113_visitPoi_cnt_180d,poi_113_visitTime_dayavg_30d,poi_113_visitTime_dayavg_90d,poi_113_visitTime_dayavg_180d,poi_100_visitDays_cnt_30d,poi_100_visitDays_cnt_90d,poi_100_visitDays_cnt_180d,poi_100_visitPoi_cnt_30d,poi_100_visitPoi_cnt_90d,poi_100_visitPoi_cnt_180d,poi_100_visitTime_dayavg_30d,poi_100_visitTime_dayavg_90d,poi_100_visitTime_dayavg_180d,poi_1506_visitDays_cnt_30d,poi_1506_visitDays_cnt_90d,poi_1506_visitDays_cnt_180d,poi_1506_visitPoi_cnt_30d,poi_1506_visitPoi_cnt_90d,poi_1506_visitPoi_cnt_180d,poi_1506_visitTime_dayavg_30d,poi_1506_visitTime_dayavg_90d,poi_1506_visitTime_dayavg_180d,poi_142_visitDays_cnt_30d,poi_142_visitDays_cnt_90d,poi_142_visitDays_cnt_180d,poi_142_visitPoi_cnt_30d,poi_142_visitPoi_cnt_90d,poi_142_visitPoi_cnt_180d,poi_142_visitTime_dayavg_30d,poi_142_visitTime_dayavg_90d,poi_142_visitTime_dayavg_180d,poi_104_visitDays_cnt_30d,poi_104_visitDays_cnt_90d,poi_104_visitDays_cnt_180d,poi_104_visitPoi_cnt_30d,poi_104_visitPoi_cnt_90d,poi_104_visitPoi_cnt_180d,poi_104_visitTime_dayavg_30d,poi_104_visitTime_dayavg_90d,poi_104_visitTime_dayavg_180d,poi_1521_visitDays_cnt_30d,poi_1521_visitDays_cnt_90d,poi_1521_visitDays_cnt_180d,poi_1521_visitPoi_cnt_30d,poi_1521_visitPoi_cnt_90d,poi_1521_visitPoi_cnt_180d,poi_1521_visitTime_dayavg_30d,poi_1521_visitTime_dayavg_90d,poi_1521_visitTime_dayavg_180d,poi_1467_visitDays_cnt_30d,poi_1467_visitDays_cnt_90d,poi_1467_visitDays_cnt_180d,poi_1467_visitPoi_cnt_30d,...,poi_10_visitTime_dayavg_180d,poi_35_visitDays_cnt_30d,poi_35_visitDays_cnt_90d,poi_35_visitDays_cnt_180d,poi_35_visitPoi_cnt_30d,poi_35_visitPoi_cnt_90d,poi_35_visitPoi_cnt_180d,poi_35_visitTime_dayavg_30d,poi_35_visitTime_dayavg_90d,poi_35_visitTime_dayavg_180d,poi_45_visitDays_cnt_30d,poi_45_visitDays_cnt_90d,poi_45_visitDays_cnt_180d,poi_45_visitPoi_cnt_30d,poi_45_visitPoi_cnt_90d,poi_45_visitPoi_cnt_180d,poi_45_visitTime_dayavg_30d,poi_45_visitTime_dayavg_90d,poi_45_visitTime_dayavg_180d,poi_25_visitDays_cnt_30d,poi_25_visitDays_cnt_90d,poi_25_visitDays_cnt_180d,poi_25_visitPoi_cnt_30d,poi_25_visitPoi_cnt_90d,poi_25_visitPoi_cnt_180d,poi_25_visitTime_dayavg_30d,poi_25_visitTime_dayavg_90d,poi_25_visitTime_dayavg_180d,poi_75_visitDays_cnt_30d,poi_75_visitDays_cnt_90d,poi_75_visitDays_cnt_180d,poi_75_visitPoi_cnt_30d,poi_75_visitPoi_cnt_90d,poi_75_visitPoi_cnt_180d,poi_75_visitTime_dayavg_30d,poi_75_visitTime_dayavg_90d,poi_75_visitTime_dayavg_180d,poi_50_visitDays_cnt_30d,poi_50_visitDays_cnt_90d,poi_50_visitDays_cnt_180d,poi_50_visitPoi_cnt_30d,poi_50_visitPoi_cnt_90d,poi_50_visitPoi_cnt_180d,poi_50_visitTime_dayavg_30d,poi_50_visitTime_dayavg_90d,poi_50_visitTime_dayavg_180d,poi_40_visitDays_cnt_30d,poi_40_visitDays_cnt_90d,poi_40_visitDays_cnt_180d,poi_40_visitPoi_cnt_30d,poi_40_visitPoi_cnt_90d,poi_40_visitPoi_cnt_180d,poi_40_visitTime_

In [155]:
score_58_24 = pd.read_csv(r"D:\work\增益分析模型\result_58_24.csv", dtype={'date': 'object'}) \
                .loc[:, ['phone', 'date', 'score_24']].rename(columns={'score_24': 'score_24_58'})
score_58_36 = pd.read_csv(r"D:\work\增益分析模型\result_58_36.csv", dtype={'date': 'object'}) \
                .loc[:, ['phone', 'date', 'score_36']].rename(columns={'score_36': 'score_36_58'})
score_byd_ym_36 = pd.read_csv(r"D:\work\增益分析模型\result_BDY_YM_36.csv", dtype={'date': 'object'}) \
                .loc[:, ['phone', 'date', 'final_score']].rename(columns={'final_score': 'score_ym_36'})
score_pp_24 = pd.read_csv(r"D:\work\增益分析模型\result_PP_24.csv", dtype={'date': 'object'}) \
                .loc[:, ['phone', 'date', 'single_pred']].rename(columns={'single_pred': 'prod_pp_24'})
score_ym_pp_24 = pd.read_csv(r"D:\work\增益分析模型\result_YM_PP_24.csv", dtype={'date': 'object'}) \
                    .loc[:, ['phone', 'date', 'final_score']].rename(columns={'final_score': 'score_ym_pp_24'})
score_cx = pd.read_csv(r"D:\work\增益分析模型\result_20250722_20w_评分_20250723_lc.csv", parse_dates=['recall_date']) \
                .rename(columns={'mobile': 'phone', 'recall_date': 'date'}).drop(['idcard', 'match_flag'], axis=1)
score_cx['date'] = score_cx['date'].apply(lambda x: str(x)[:10])

score_ali_1 = pd.read_csv(r"D:\work\增益分析模型\59378_56b91_256-1_anti_fraud_v6_score.csv", parse_dates=['applydate']) \
                .rename(columns={'mobileSha256': 'phone', 'applydate': 'date', 'score': 'score_ali'}) \
                .loc[:, ['phone', 'date', 'score_ali']]
score_ali_2 = pd.read_csv(r"D:\work\增益分析模型\59380_18b1b_md5-1_anti_fraud_v6_score.csv", parse_dates=['applydate']) \
                .rename(columns={'mobileMd5': 'phone', 'applydate': 'date', 'score': 'score_ali'}) \
                .loc[:, ['phone', 'date', 'score_ali']]
score_ali = pd.concat([score_ali_1, score_ali_2], ignore_index=True)
score_ali['date'] = score_ali['date'].apply(lambda x: str(x)[:10])
score_ali['score_ali'] = score_ali['score_ali'].apply(lambda x: None if x == -1 else x)

score_tx_1 = pd.read_excel(r"D:\work\增益分析模型\反欺诈v8s_行业风险4.1_风险分层v4-20250721-4w.xlsx", parse_dates=['app_date']) \
                .rename(columns={'phonenum_sha256': 'phone', 'app_date': 'date'}) \
                .loc[:, ['phone', 'date', 'fx_score_v04', 'model_risk_v8_shighirr_score', 'model_risk_v8_slowirr_score', 'model_risk_v8_stongyong_score']]
score_tx_2 = pd.read_excel(r"D:\work\增益分析模型\反欺诈v8s_行业风险4.1_风险分层v4-20250721-16w.xlsx", parse_dates=['app_date']) \
                .rename(columns={'phonenum_md5': 'phone', 'app_date': 'date'}) \
                .loc[:, ['phone', 'date', 'fx_score_v04', 'model_risk_v8_shighirr_score', 'model_risk_v8_slowirr_score', 'model_risk_v8_stongyong_score']]
score_tx = pd.concat([score_tx_1, score_tx_2], ignore_index=True)
score_tx['date'] = score_tx['date'].apply(lambda x: str(x)[:10])



In [239]:
print(f"score_ali shape: {score_ali.shape}")
print(f"score_fn shape: {score_fn.shape}")

score_ali shape: (200000, 3)
score_fn shape: (83792, 5)


In [255]:
score_ali

,phone,date,score_ali
0,f241a6dc833fa087a49fac4c2f19dd4e4dbae2a6b8e3a8...,2023-09-06,82.33
1,ee0881af1a596bae87a55473e5b735cc7334e9d5a3bb1a...,2024-01-13,36.14
2,ac7422965f26c8328e1149f3a4464c5fbc38d47ef076ab...,2023-11-23,26.06
3,2acfae7a97a1838114ee9c08f84954ece073e3e2e4bda7...,2023-09-27,12.93
4,ac36aa79c9abe7a90d9d79478d9af4ccf90edf6d686eb3...,2023-09-05,71.75
...,...,...,...
199995,d7b7b9ce8e71f623fb3eb917aee8c257,2024-10-06,99.41
199996,9c7374eef1d9bbfff83065e5aa3c4337,2024-10-06,99.53
199997,8ccc59748aceeb80bdc508bd0ae1b531,2024-10-04,80.65
199998,a6be2fb5d686c9e46c0c61439b96b6d9,2024-10-02,99.81


In [249]:
merged_ali.head()

,phone,date,score_ali,poi_20_visitTime_dayavg_90d,poi_1475_visitDays_cnt_180d,poi_20_visitDays_cnt_180d
0,f241a6dc833fa087a49fac4c2f19dd4e4dbae2a6b8e3a8...,2023-09-06,82.33,NaN,NaN,NaN
1,ee0881af1a596bae87a55473e5b735cc7334e9d5a3bb1a...,2024-01-13,36.14,5451.0,2.0,42.0
2,ac7422965f26c8328e1149f3a4464c5fbc38d47ef076ab...,2023-11-23,26.06,NaN,NaN,NaN
3,2acfae7a97a1838114ee9c08f84954ece073e3e2e4bda7...,2023-09-27,12.93,NaN,NaN,NaN
4,ac36aa79c9abe7a90d9d79478d9af4ccf90edf6d686eb3...,2023-09-05,71.75,NaN,NaN,NaN


In [248]:
merged_ali = pd.merge(score_ali, score_fn, on=['phone','date'], how='left')

In [254]:
matched_count = merged_ali['poi_20_visitTime_dayavg_90d'].notna().sum()
print(f"与 score_fn 成功匹配的样本数: {matched_count}")

与 score_fn 成功匹配的样本数: 13827


In [258]:
print(f"score_tx shape: {score_ali.shape}")
print(f"score_fn shape: {score_fn.shape}")

score_tx shape: (200000, 3)
score_fn shape: (83792, 5)


In [257]:
merged_tx=score_tx.merge(score_fn,on=['phone','date'],how='left')

In [259]:
matched_tx = merged_tx['poi_20_visitTime_dayavg_90d'].notna().sum()
print(f"与 score_fn 成功匹配的样本数: {matched_tx}")

与 score_fn 成功匹配的样本数: 13827


In [213]:
features=['poi_20_visitTime_dayavg_90d',
          'poi_1475_visitDays_cnt_180d',
         'poi_20_visitDays_cnt_180d']
non_feature_cols = ['date','phone']
columns= non_feature_cols + features

In [214]:
score_fn = score_fn[columns]

In [215]:
score_fn.columns

Index(['date', 'phone', 'poi_20_visitTime_dayavg_90d', 'poi_1475_visitDays_cnt_180d', 'poi_20_visitDays_cnt_180d'], dtype='object')

In [216]:
# 把所有相关 dataframe 的 date 列统一转换为 datetime 类型
for df in [target_all, score_58_24, score_58_36, score_byd_ym_36, 
           score_pp_24, score_ym_pp_24, score_cx, score_ali, score_tx,score_fn]:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')


In [217]:
score_all = target_all.merge(score_58_24, on=['phone', 'date'], how='left') \
                        .merge(score_58_36, on=['phone', 'date'], how='left') \
                        .merge(score_byd_ym_36, on=['phone', 'date'], how='left') \
                        .merge(score_pp_24, on=['phone', 'date'], how='left') \
                        .merge(score_ym_pp_24, on=['phone', 'date'], how='left') \
                        .merge(score_cx, on=['phone', 'date'], how='left') \
                        .merge(score_ali, on=['phone', 'date'], how='left') \
                        .merge(score_tx, on=['phone', 'date'], how='left') \
                        .merge(score_fn, on=['phone', 'date'], how='left') \
                        .assign(weight=1) \
                        .rename(columns={'score_24_58':'峦盾分V7','score_36_58':'峦盾分V8','score_ym_36':'川峰分V1','score_ym_pp_24':'川衡分V2','':''})
score_all.drop_duplicates('phone', inplace=True)

In [218]:
start_col = score_all.columns.get_loc('峦盾分V7')
score_part = score_all.iloc[:,start_col:]
score_all = pd.concat([score_all[['customer','target']], score_part], axis=1)

In [220]:
score_all.isna().sum()

customer                              0
target                                0
峦盾分V7                            208005
峦盾分V8                            208005
川峰分V1                            151858
prod_pp_24                       350918
川衡分V2                            111266
BZ_R1V2_Scores                   394313
DR_R1_Scores                     394313
QZ_R1_Scores                     394313
score_ali                        335000
fx_score_v04                     334633
model_risk_v8_shighirr_score     334553
model_risk_v8_slowirr_score      334420
model_risk_v8_stongyong_score    334553
poi_20_visitTime_dayavg_90d      404598
poi_1475_visitDays_cnt_180d      404598
poi_20_visitDays_cnt_180d        404598
weight                                0
dtype: int64

In [221]:
score_all.groupby('customer').apply(lambda x: x.notna().mean())

,customer,target,峦盾分V7,峦盾分V8,川峰分V1,prod_pp_24,川衡分V2,BZ_R1V2_Scores,DR_R1_Scores,QZ_R1_Scores,score_ali,fx_score_v04,model_risk_v8_shighirr_score,model_risk_v8_slowirr_score,model_risk_v8_stongyong_score,poi_20_visitTime_dayavg_90d,poi_1475_visitDays_cnt_180d,poi_20_visitDays_cnt_180d,weight
customer,,,,,,,,,,,,,,,,,,,
DD,1.0,1.0,0.522874,0.522874,0.591794,0.311617,0.713446,0.138829,0.138829,0.138829,0.198109,0.199091,0.199200,0.199320,0.199200,0.151086,0.151086,0.151086,1.0
HR,1.0,1.0,0.529868,0.529868,0.671987,0.251111,0.750979,0.085530,0.085530,0.085530,0.169483,0.169543,0.169603,0.169761,0.169603,0.245094,0.245094,0.245094,1.0
NB,1.0,1.0,0.822793,0.822793,0.961365,0.289686,0.970306,0.553331,0.553331,0.553331,0.998489,0.998917,0.999144,0.999849,0.999144,0.000000,0.000000,0.000000,1.0
YG,1.0,1.0,0.811994,0.811994,0.946252,0.319441,0.957944,0.700466,0.700466,0.700466,0.993247,0.997379,0.998337,0.999521,0.998337,0.000000,0.000000,0.000000,1.0


In [88]:
score_all.columns

Index(['customer', '峦盾分V7', '峦盾分V8', '川峰分V1', 'prod_pp_24', '川衡分V2', 'BZ_R1V2_Scores', 'DR_R1_Scores', 'QZ_R1_Scores', 'score_ali', 'fx_score_v04', 'model_risk_v8_shighirr_score', 'model_risk_v8_slowirr_score', 'model_risk_v8_stongyong_score', 'encrypt_y', 'uuid', 'poi_20_visitTime_dayavg_90d', 'poi_1475_visitDays_cnt_180d', 'poi_20_visitDays_cnt_180d', 'weight'], dtype='object')

In [223]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def calculate_ks(y_true, y_score, n_bins=10):
    # 强制转换为数值型，遇到异常转为 NaN
    y_true = pd.to_numeric(y_true, errors='coerce')
    y_score = pd.to_numeric(y_score, errors='coerce')

    df = pd.DataFrame({'score': y_score, 'bad': y_true})
    df.dropna(inplace=True)
    df = df.sort_values('score')

    df['bucket'] = pd.qcut(df['score'], q=n_bins, duplicates='drop')
    grouped = df.groupby('bucket', as_index=False)
    result = grouped.agg({'score': ['min', 'max', 'count'], 'bad': ['sum', 'mean']})
    result.columns = ['_'.join(col).strip() for col in result.columns.values]

    result = result.rename(columns={
        'score_min': 'min_score',
        'score_max': 'max_score',
        'score_count': 'total',
        'bad_sum': 'bad_count',
        'bad_mean': 'bad_rate'
    })

    # 再次强制转换
    result['total'] = pd.to_numeric(result['total'], errors='coerce')
    result['bad_count'] = pd.to_numeric(result['bad_count'], errors='coerce')

    result['good_count'] = result['total'] - result['bad_count']
    result['cum_good'] = result['good_count'].cumsum()
    result['cum_bad'] = result['bad_count'].cumsum()
    result['cum_good_pct'] = result['cum_good'] / result['good_count'].sum()
    result['cum_bad_pct'] = result['cum_bad'] / result['bad_count'].sum()
    result['ks'] = np.abs(result['cum_bad_pct'] - result['cum_good_pct'])

    return result['ks'].max() * 100



score_col = score_all.loc[:, '峦盾分V7':].drop(['prod_pp_24','weight'],axis=1).columns.tolist()
cust_ks = pd.DataFrame(index=score_col)
for cust in ['DD','HR']:
    print(cust)
    cust_fn = pd.DataFrame()
    for col in score_col:
        score_cust = score_all.loc[score_all['customer'].eq(cust), ['target', col]].copy().dropna()
        df_train = score_cust.sample(frac=0.7, random_state=2)
        df_test = score_cust.drop(df_train.index)  
        
        cust_fn.at[col, cust+'_train_ks'] = calculate_ks(df_train['target'], df_train[col])
        cust_fn.at[col, cust+'_test_ks'] = calculate_ks(df_test['target'], df_test[col])  
    cust_ks = cust_ks.join(cust_fn)
cust_ks

DD
HR


,DD_train_ks,DD_test_ks,HR_train_ks,HR_test_ks
峦盾分V7,9.390731,6.721742,5.782440,5.135494
峦盾分V8,4.976624,3.128321,2.061882,2.272116
川峰分V1,6.103769,6.129142,2.889134,4.374387
川衡分V2,14.158192,15.340242,14.275688,13.247020
BZ_R1V2_Scores,4.997864,4.936970,3.462408,4.345261
DR_R1_Scores,8.677265,14.125927,5.110093,3.210432
QZ_R1_Scores,8.815688,10.415289,3.813264,3.907234
score_ali,19.266465,20.605817,15.730721,17.258615
fx_score_v04,14.573182,16.382502,14.245578,14.640603
model_risk_v8_shighirr_score,20.294057,18.786850,12.991068,15.317429


In [97]:
score_base

['峦盾分V7',
 '峦盾分V8',
 '川峰分V1',
 '川衡分V2',
 'BZ_R1V2_Scores',
 'DR_R1_Scores',
 'QZ_R1_Scores',
 'score_ali',
 'fx_score_v04',
 'model_risk_v8_shighirr_score',
 'model_risk_v8_slowirr_score',
 'model_risk_v8_stongyong_score',
 'encrypt_y',
 'uuid']

In [231]:
from hundredim.wrapper import combiner
import math
import pandas as pd

# 定义特征列表
feature_dict = {
    'DD': ['poi_20_visitTime_dayavg_90d', 'poi_1475_visitDays_cnt_180d'],
    'HR': ['poi_20_visitDays_cnt_180d']
}

# 所有的候选 base 特征
score_base = score_all.loc[:, '峦盾分V7':].drop(['prod_pp_24', 'weight'], axis=1).columns.tolist()
score_base = [x for x in score_base if x not in sum(feature_dict.values(), [])]  # 排除入模特征

def calculate_ks(y_true, y_pred):
    from sklearn.metrics import roc_curve
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    return max(tpr - fpr)

# 遍历每个客户类型
for cust, feature_list in feature_dict.items():
    zy_result = pd.DataFrame()

    for s1 in feature_list:
        print(f"【{cust}】正在处理特征：{s1}")
        score_cust = score_all[score_all['customer'].eq(cust) & score_all[s1].notna()].copy()

        if score_cust.empty:
            print(f"❌ 客群 {cust} 的特征 {s1} 没有有效数据，跳过")
            continue

        df_train = score_cust.sample(frac=0.7, random_state=2)
        df_test = score_cust.drop(df_train.index)

        for s2 in score_base:
            if s2 not in df_train.columns:
                print(f"❌ 缺失列：{s2}")
                continue

            if df_train[s2].dropna().empty:
                print(f"⚠️ 特征 {s2} 在训练集中全是空，跳过")
                continue

            score_new = [s1, s2]
            cbn = combiner.Combiner()
            cbn.process(df_train, score_new, 'target', 'weight', base_rate=1, pdo=math.log(2), base_score=0)
            combine_all = cbn.combination(df_train, df_test, score_new, 'target', 'weight')

            # KS 增益
            combine_all['fn_ks_train'] = calculate_ks(df_train['target'], df_train[s1])
            combine_all['fn_ks_test'] = calculate_ks(df_test['target'], df_test[s1])
            combine_all['base_ks_train'] = calculate_ks(df_train['target'], df_train[s2].fillna(-1))
            combine_all['base_ks_test'] = calculate_ks(df_test['target'], df_test[s2].fillna(-1))

            combine_all[['ks_train', 'ks_test']] = combine_all[['ks_train', 'ks_test']] * 100
            combine_all['train_ks_zy'] = combine_all['ks_train'] - combine_all['base_ks_train']
            combine_all['test_ks_zy'] = combine_all['ks_test'] - combine_all['base_ks_test']

            combine_all_less = combine_all[[
                'subscores_and_weights_std',
                'ks_train', 'ks_test',
                'fn_ks_train', 'fn_ks_test',
                'base_ks_train', 'base_ks_test',
                'train_ks_zy', 'test_ks_zy'
            ]]
            combine_all_less = combine_all_less.rename(columns=lambda col: f"{col}_{cust}")
            combine_all_less = combine_all_less.round(2)
            zy_result = pd.concat([zy_result, combine_all_less], ignore_index=True)

    if not zy_result.empty:
        output_path = fr'D:\work\测评\飞鸟\增益分析\fn_combine_{cust}.xlsx'
        zy_result.to_excel(output_path, index=False)
        print(f"✅ 文件已保存至：{output_path}")
    else:
        print(f"⚠️ 客群 {cust} 无有效分析结果") 

【DD】正在处理特征：poi_20_visitTime_dayavg_90d
开始计算rescaling以及填充，使用的特殊值为[]以及nan
填充: 峦盾分V7_rescaled nan -3.1366721494465706
填充: 峦盾分V7_rescaled nan -3.1366721494465706


100%|██████████| 1/1 [00:00<00:00,  6.30it/s]


开始计算rescaling以及填充，使用的特殊值为[]以及nan
填充: 峦盾分V8_rescaled nan -3.136672149446752
填充: 峦盾分V8_rescaled nan -3.136672149446752


100%|██████████| 1/1 [00:00<00:00,  7.10it/s]


开始计算rescaling以及填充，使用的特殊值为[]以及nan
填充: 川峰分V1_rescaled nan -3.0033697006831575
填充: 川峰分V1_rescaled nan -3.0033697006831575


...

开始计算rescaling以及填充，使用的特殊值为[]以及nan
填充: model_risk_v8_slowirr_score_rescaled nan -2.9315949065112474
填充: model_risk_v8_slowirr_score_rescaled nan -2.9315949065112474


100%|██████████| 1/1 [00:00<00:00,  4.60it/s]


开始计算rescaling以及填充，使用的特殊值为[]以及nan
填充: model_risk_v8_stongyong_score_rescaled nan -2.9317183061721117
填充: model_risk_v8_stongyong_score_rescaled nan -2.9317183061721117


100%|██████████| 1/1 [00:00<00:00,  3.98it/s]

✅ 文件已保存至：D:\work\测评\飞鸟\增益分析\fn_combine_HR.xlsx
